In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.utils as vutils
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np


In [45]:

import torchvision
from torchvision import transforms

def load_dataset(folder_path):
    transform = transforms.Compose([
        transforms.Resize((32, 32)),  # Resize images to 32x32
        transforms.ToTensor(),       # Convert images to tensors
        transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
    ])
    dataset = torchvision.datasets.ImageFolder(folder_path, transform=transform)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)
    return dataloader

In [59]:
# Define the Generator
class Generator(nn.Module):
    def __init__(self, latent_dim=100):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 3 * 32 * 32),  # Output 3-channel image (RGB)
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        return img.view(-1, 3, 32, 32)  # Reshape to (batch, 3, 32, 32)


# Define the Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(3 * 32 * 32, 512),  # Adjust input size to match flattened image size (3072)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)  # Flatten the image
        return self.model(img_flat)

In [60]:
# Train the GAN
def train_gan(dataloader, num_epochs=100, latent_dim=100, lr=0.0002):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    generator = Generator(latent_dim).to(device)
    discriminator = Discriminator().to(device)

    loss_function = nn.BCELoss()
    optimizer_G = optim.Adam(generator.parameters(), lr=lr)
    optimizer_D = optim.Adam(discriminator.parameters(), lr=lr)

    for epoch in range(num_epochs):
        for i, (real_images, _) in enumerate(dataloader):
            real_images = real_images.to(device)
            batch_size = real_images.size(0)

            # Labels
            real_labels = torch.ones(batch_size, 1).to(device)
            fake_labels = torch.zeros(batch_size, 1).to(device)

            # Train Discriminator
            optimizer_D.zero_grad()
            outputs = discriminator(real_images)
            loss_real = loss_function(outputs, real_labels)

            z = torch.randn(batch_size, latent_dim).to(device)
            fake_images = generator(z)
            outputs = discriminator(fake_images.detach())
            loss_fake = loss_function(outputs, fake_labels)

            loss_D = loss_real + loss_fake
            loss_D.backward()
            optimizer_D.step()

            # Train Generator
            optimizer_G.zero_grad()
            outputs = discriminator(fake_images)
            loss_G = loss_function(outputs, real_labels)
            loss_G.backward()
            optimizer_G.step()

        print(f"Epoch [{epoch+1}/{num_epochs}] | Loss D: {loss_D.item()} | Loss G: {loss_G.item()}")

        if (epoch + 1) % 10 == 0:
            vutils.save_image(fake_images, f"generated_epoch_{epoch+1}.png", normalize=True)

    torch.save(generator.state_dict(), "generator.pth")


In [ ]:
# Generate New Images
def generate_images(generator_path, num_images=5, latent_dim=100):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    generator = Generator(latent_dim).to(device)
    generator.load_state_dict(torch.load(generator_path, map_location=device))
    generator.eval()

    z = torch.randn(num_images, latent_dim).to(device)
    with torch.no_grad():
        fake_images = generator(z)

    for i, img in enumerate(fake_images):
        img = img.cpu().numpy().transpose(1, 2, 0)  # Reshape to (H, W, C)
        img = ((img + 1) / 2 * 255).astype(np.uint8)  # Normalize to [0,255]

        # Ensure RGB mode
        if img.shape[-1] == 1:  # Grayscale
            img = img.squeeze(-1)  # Remove channel dim
            Image.fromarray(img, mode="L").save(f"generated_{i}.png")
        else:  # RGB Image
            Image.fromarray(img).save(f"generated_{i}.png")

    print(f"Generated {num_images} images and saved them.")


In [62]:
folder_path = r"D:\college\imp-doc\sem6\GENAI\project\new"
dataloader = load_dataset(folder_path)
train_gan(dataloader)
generate_images("generator.pth")

Epoch [1/100] | Loss D: 0.3445831835269928 | Loss G: 1.5341498851776123
Epoch [2/100] | Loss D: 0.5701174736022949 | Loss G: 1.9194046258926392
Epoch [3/100] | Loss D: 0.08145958930253983 | Loss G: 3.887817144393921
Epoch [4/100] | Loss D: 0.39373141527175903 | Loss G: 1.7034183740615845
Epoch [5/100] | Loss D: 0.9711621403694153 | Loss G: 0.6926404237747192
Epoch [6/100] | Loss D: 0.25078630447387695 | Loss G: 2.7622838020324707
Epoch [7/100] | Loss D: 0.04595518857240677 | Loss G: 6.106468677520752
Epoch [8/100] | Loss D: 0.1286110132932663 | Loss G: 2.4408419132232666
Epoch [9/100] | Loss D: 0.2039618194103241 | Loss G: 4.892741680145264
Epoch [10/100] | Loss D: 0.15970689058303833 | Loss G: 3.4430363178253174
Epoch [11/100] | Loss D: 0.006494171917438507 | Loss G: 7.593844890594482
Epoch [12/100] | Loss D: 0.06160243600606918 | Loss G: 3.615633964538574
Epoch [13/100] | Loss D: 0.019548453390598297 | Loss G: 5.83271598815918
Epoch [14/100] | Loss D: 0.060220278799533844 | Loss G: 4

C:\Users\sameer\AppData\Local\Temp\ipykernel_33180\4135465067.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  generator.load_state_dict(torch.load(generator_path, map_lo

TypeError: Cannot handle this data type: (1, 1, 32), |u1

In [65]:
import torch
import torchvision.utils as vutils

# Generate a batch of images
latent_dim = 100
generator = Generator(latent_dim)  # Ensure the generator is loaded
z = torch.randn(1, latent_dim)  # Generate a random noise vector
generated_img = generator(z)

# Save the image
vutils.save_image(generated_img, "generated_image.png", normalize=True)
print("Image saved as 'generated_image.png'")


Image saved as 'generated_image.png'
